In [2]:
import torch
import time
import psutil
from tqdm import tqdm

from transformers import  WhisperForConditionalGeneration, WhisperProcessor
from datasets import load_dataset
from collections import defaultdict
from evaluate import load

c:\Users\orlov\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MODEL_NAME = "openai/whisper-small"  # 'openai/whisper-large-v2'

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
processor = WhisperProcessor.from_pretrained(MODEL_NAME)
model

cuda


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0): WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

### model size

In [5]:
model_size = sum(p.numel() for p in model.parameters()) * 4 / (1024 ** 2) 
print(f"Model size: {model_size} MB")

Model size: 922.1455078125 MB


### inference time

In [6]:
s = 30
sample_audio = torch.randn(1, 16000 * s).squeeze().numpy()
input_features = processor(sample_audio, sampling_rate=16000, return_tensors="pt").input_features
input_features

tensor([[[1.1365, 1.0739, 0.6812,  ..., 1.0912, 1.2428, 1.1397],
         [1.2856, 0.9602, 1.0401,  ..., 0.9131, 1.2737, 1.2955],
         [1.3244, 1.1136, 1.2464,  ..., 0.8805, 1.1707, 1.2634],
         ...,
         [1.1176, 1.2048, 1.1087,  ..., 1.0988, 1.1286, 1.1829],
         [1.1768, 1.1551, 1.0370,  ..., 1.1124, 1.0675, 1.1139],
         [1.1641, 1.1053, 1.1308,  ..., 1.1202, 1.1815, 1.1866]]])

In [7]:
def measure_inference_time(model, input_tensor, device):
    model.to(device)
    input_tensor = input_tensor.to(device)
    
    # Замеряем время
    start_time = time.time()
    with torch.no_grad():
        output = model.generate(input_tensor, return_timestamps=True)[0]
    processor.decode(output)

    elapsed_time = (time.time() - start_time) * 1000  # Время в миллисекундах
    return round(elapsed_time, 2)

# Запуск теста на CPU и GPU
cpu_time = measure_inference_time(model, input_features.cpu(), "cpu")
gpu_time = measure_inference_time(model, input_features, "cuda") if torch.cuda.is_available() else "N/A"

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [8]:
cpu_time / s, gpu_time /s

(1581.0386666666668, 593.903)

### Замер использования RAM

In [11]:
ram_usage = psutil.Process().memory_info().rss

model.to('cpu')
ram_usage_full = psutil.Process().memory_info().rss

ram_usage = round((ram_usage_full - ram_usage) / (1024 ** 2), 2)
ram_usage

922.8

### Замер использования VRAM

In [9]:
vram_usage = round(torch.cuda.memory_allocated() / (1024 ** 2), 2) if torch.cuda.is_available() else "N/A"
vram_usage

931.01

### Оценка качества (CER, WER)

In [15]:
dataset = load_dataset("librispeech_asr", "clean", split="test")
# dataset = load_dataset("librispeech_asr", "test.clean")

Generating test split: 100%|██████████| 2620/2620 [00:01<00:00, 1645.37 examples/s]


In [27]:
wer = load("wer")
cer = load("cer")

def predict(batch, model):
    audio = batch["audio"]
    input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features

    with torch.no_grad():
        predicted_ids = model.generate(input_features.to("cuda"))[0]
    transcription = processor.decode(predicted_ids)
    return processor.tokenizer._normalize(transcription)

In [28]:
NUM_EXAMPLES = 64
res = defaultdict(list)

for el in tqdm(dataset.select(range(NUM_EXAMPLES))):
    res["reference"].append(processor.tokenizer._normalize(el['text']))
    res["prediction"].append(predict(el, model))

cer_res = 100 * cer.compute(references=res["reference"], predictions=res["prediction"])
wer_res = 100 * wer.compute(references=res["reference"], predictions=res["prediction"])

100%|██████████| 64/64 [00:23<00:00,  2.68it/s]


In [30]:
# Вывод результатов
results = {
    "Модель ": MODEL_NAME,
    "Метод ": "Whisper",
    "Размер весов (MB) ": round(model_size, 2),
    "Время инференса (CPU, ms) ": cpu_time / s,
    "Время инференса (GPU, ms) ": gpu_time / s,
    "Использование RAM (MB) ": ram_usage,
    "Использование VRAM (MB) ": vram_usage,
    "Качество ": {
        "CER% ": cer_res,
        "WER% ": wer_res,
    }
}

results

{'Модель ': 'openai/whisper-small',
 'Метод ': 'Whisper',
 'Размер весов (MB) ': 922.15,
 'Время инференса (CPU, ms) ': 377.64500000000004,
 'Время инференса (GPU, ms) ': 156.31666666666666,
 'Использование RAM (MB) ': 2385.38,
 'Использование VRAM (MB) ': 931.01,
 'Качество ': {'CER% ': 1.3379017784304128, 'WER% ': 3.829787234042553}}